In [ ]:
import pandas as _hex_pandas
import datetime as _hex_datetime
import json as _hex_json

In [ ]:
hex_scheduled = _hex_json.loads("false")

In [ ]:
hex_user_email = _hex_json.loads("\"example-user@example.com\"")

In [ ]:
hex_run_context = _hex_json.loads("\"logic\"")

In [ ]:
hex_timezone = _hex_json.loads("\"UTC\"")

In [ ]:
hex_project_id = _hex_json.loads("\"ec766d10-159d-4685-bbbe-d32d55256000\"")

In [ ]:
hex_project_name = _hex_json.loads("\"Signature Extractions from Emails using LLMs\"")

In [ ]:
hex_status = _hex_json.loads("\"\"")

In [ ]:
hex_categories = _hex_json.loads("[]")

In [ ]:
hex_color_palette = _hex_json.loads("[\"#4C78A8\",\"#F58518\",\"#E45756\",\"#72B7B2\",\"#54A24B\",\"#EECA3B\",\"#B279A2\",\"#FF9DA6\",\"#9D755D\",\"#BAB0AC\"]")

In this notebook, I show my working process on how to extract signature details from various scenario emails using LLMs.

I will be comparing the `claude-3.5` and `gpt-3.5` models.

I have _13 different emails_ scenarios and a way to score which model performs best.

My scoring function compares three things:

- Does the LLM extract the required fields specified?
- Does the LLM extract field match what is expected from the test case?
- Does the LLM extract the data in the required specified format?

Among the three, the third one is most imporant since the data could be an input to another system and wrong formatted data would break the functioning of the said system.

In [ ]:
import re
import json
import time
import requests
import logging

import tiktoken

In [ ]:
def clean_json_string(json_str):
    # Replace single quotes with double quotes
    json_str = json_str.replace("'", '"')
    
    # Handle "NULL" values (making sure they are correctly replaced without extra quotes)
    # json_str = re.sub(r'""NULL""', '"NULL"', json_str)
    json_str = re.sub(r'(\bNULL\b)', '"NULL"', json_str)
    json_str = re.sub(r'""NULL""', '"NULL"', json_str)
    
    # Remove any trailing commas in objects and arrays
    json_str = re.sub(r',\s*}', '}', json_str)
    json_str = re.sub(r',\s*\]', ']', json_str)
    
    return json_str

def ask_open_ai(system_settings, ask):
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
    "Content-Type": "application/json", 
    "Authorization": "Bearer ",

    }
    data = {
        "model": "gpt-3.5-turbo",
        "messages": [
            {"role": "system", "content": system_settings},
            {"role": "user", "content": [{"type": "text", "text": ask}]},
        ],
        "temperature": 0.0
        }

    response = requests.post(url, headers=headers, data=json.dumps(data))
    content = response.json()['choices'][0]['message']['content']

    try:
        return json.loads(content)
    except json.JSONDecodeError:
        return json.loads(clean_json_string(content))


def ask_claude(system_settings, ask):
    url = "https://api.anthropic.com/v1/messages"
    headers = {
        "content-type": "application/json",
        "x-api-key": "",
        "anthropic-version": "2023-06-01"
    }
    data = {
        "model": "claude-3-sonnet-20240229",
        "system": system_settings,
        "messages": [
            {"role": "user", "content": ask}
        ],
        "max_tokens": 1024,
        "temperature": 0.0
    }
    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()
        
        response_data = response.json()
        
        if 'content' in response_data:
            content = response_data['content'][0]['text']
            try:
                content = clean_json_string(content)
                return json.loads(content)
            except json.JSONDecodeError:
                print("Content is not a valid JSON string. Returning as is.")
                return content
        else:
            print("Unexpected response structure:", response_data)
            return None
    
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
        print(f"Response content: {response.text}")
    except json.JSONDecodeError as json_err:
        print(f"JSON decode error: {json_err}")
        print(f"Raw response: {response.text}")
    except Exception as err:
        print(f"An error occurred: {err}")
        print(f"Raw response: {response.text}")

In [ ]:
email_content = """
Hi Team,

Please find attached the latest project report.

Thanks,
Emily Johnson
Marketing Coordinator
emily.j@techstart.io
"""

In [ ]:
def generate_prompts(email_content_to_extract):
    response_structure = {
    "__sig__": {
        "first_name": "Jane",
        "second_name": "Doe",
        "company": "ABC Company",
        "position": "Sales & Marketing Director",
        "phone": "(800) 555-1133",
        "email": "jane.doe@example.com",
        "website": "www.example.com",
        }
    }

    prompt = f"""
    You are an expert agent in data analysis and natural language processing. Your task is to process a set of email texts and extract the email signature information. Please adhere strictly to the following instructions and return the information in the specified format only:

    TASK: Extract the email signature from the following email and structure it into a JSON format.

    Instructions:
    1. Identify the signature block within the email text.
    2. Extract the following fields from the signature: first_name, second_name, company, position, phone, email, and website.
    3. If a field is missing, return 'NULL' in the JSON structure.
    4. Ensure that the extracted information is accurate and consistent.
    5. Adhere strictly to the format provided below and only return the information as requested.
    6. Do not include any additional text or information beyond the required JSON structure.
    7. Be smart about what information you can or cannot extract according to the context
    8. In case of multiple details in an email, extract data from the relevant data only

    Email text to process:
    -----------------------------------------
    {email_content_to_extract}
    -----------------------------------------
    """

    return_format = f"""
    Your response must be in valid JSON format.
    Wrap the entire response in a JSON object with a single key '__sig__'. containing the extracted signature fields.
    Example structure provided below. Do not return any other text information apart from the response JSON requested:

    -----------------------------------------

    {response_structure}
    """

    return prompt, return_format

In [ ]:
# Load the encoding for the model you are using
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

def count_tokens(text):
    # Encode the text into tokens
    tokens = encoding.encode(text)
    # Count the number of tokens
    token_count = len(tokens)

    return token_count

# Example usage:
prompt, return_format = generate_prompts(email_content)

print(prompt)
print('\n')
print(return_format)
number_of_tokens = count_tokens(prompt + prompt + return_format)
print(f"The text contains {number_of_tokens} tokens.")


    You are an expert agent in data analysis and natural language processing. Your task is to process a set of email texts and extract the email signature information. Please adhere strictly to the following instructions and return the information in the specified format only:

    TASK: Extract the email signature from the following email and structure it into a JSON format.

    Instructions:
    1. Identify the signature block within the email text.
    2. Extract the following fields from the signature: first_name, second_name, company, position, phone, email, and website.
    3. If a field is missing, return 'NULL' in the JSON structure.
    4. Ensure that the extracted information is accurate and consistent.
    5. Adhere strictly to the format provided below and only return the information as requested.
    6. Do not include any additional text or information beyond the required JSON structure.
    7. Be smart about what information you can or cannot extract according to the co

In [ ]:
email_content_open_ai = ask_open_ai(
    system_settings = return_format,
    ask = prompt,
)

email_content_anthropic = ask_claude(
    system_settings = return_format,
    ask = prompt,
)

In [ ]:
print(email_content_open_ai)

{'__sig__': {'first_name': 'Emily', 'second_name': 'Johnson', 'company': 'NULL', 'position': 'Marketing Coordinator', 'phone': 'NULL', 'email': 'emily.j@techstart.io', 'website': 'NULL'}}


In [ ]:
print(email_content_anthropic)

{'__sig__': {'first_name': 'Emily', 'second_name': 'Johnson', 'company': 'NULL', 'position': 'Marketing Coordinator', 'phone': 'NULL', 'email': 'emily.j@techstart.io', 'website': 'NULL'}}


In [ ]:
import json
from difflib import SequenceMatcher

def calculate_similarity(response_value, expected_value):
    return SequenceMatcher(None, str(response_value).lower(), str(expected_value).lower()).ratio()

def evaluate_response(response, test_case):
    expected_fields = ["first_name", "second_name", "company", "position", "phone", "email", "website"]
    max_score = len(expected_fields)

    try:
        if isinstance(response, str):
            try:
                response_json = json.loads(response)
            except json.JSONDecodeError:
                return {
                    "quality": 0,
                    "score": 0,
                    "max_score": max_score,
                    "details": ["Error: Response is not a valid JSON string"],
                    "similarities": []
                }
        elif isinstance(response, dict):
            response_json = response
        else:
            return {
                "quality": 0,
                "score": 0,
                "max_score": max_score,
                "details": ["Error: Response is neither a string nor a dictionary"],
                "similarities": []
            }

        response_json = response_json.get('__sig__', response_json)

        score = 0
        details = []
        similarities = []

        for field in expected_fields:
            response_value = response_json.get(field, "").strip()
            expected_value = test_case.get(field, "")

            if response_value.lower() == "null" and expected_value.lower() == "null":
                score += 1
                details.append(f"{field}: Present (NULL matched)")
                similarities.append(1)  # Treat as perfect match
            elif response_value and response_value.lower() != "null":
                score += 1
                details.append(f"{field}: Present")
                similarity = calculate_similarity(response_value, expected_value)
                similarities.append(similarity)
            else:
                details.append(f"{field}: Missing or NULL")
                similarities.append(0)

        quality = (score / max_score) * 100

        return {
            "quality": round(quality, 2),
            "score": score,
            "max_score": max_score,
            "details": details,
            "similarities": similarities
        }

    except Exception as e:
        return {
            "quality": 0,
            "score": 0,
            "max_score": max_score,
            "details": [f"Error in evaluation: {str(e)}"],
            "similarities": []
        }


In [ ]:
expected_response = {
        "first_name": "Emily",
        "second_name": "Johnson",
        "company": "NULL",
        "position": "Marketing Coordinator",
        "phone": "NULL",
        "email": "emily.j@techstart.io",
        "website": "NULL"
    }

gpt_score = evaluate_response(email_content_open_ai, expected_response)
gpt_similarity = sum(gpt_score['similarities']) / len(gpt_score['similarities']) * 100 if gpt_score['similarities'] else 0
claude_score = evaluate_response(email_content_anthropic, expected_response)
claude_similarity = sum(claude_score['similarities']) / len(claude_score['similarities']) * 100 if claude_score['similarities'] else 0

print(f"GPT 3.5 Score: Quality = {gpt_score['quality']}%, Similarity: {gpt_similarity}%")
print(f"Claude 3.5 Score: Quality = {claude_score['quality']}%, Similarity: {claude_similarity}%")

GPT 3.5 Score: Quality = 100.0%, Similarity: 100.0%
Claude 3.5 Score: Quality = 100.0%, Similarity: 100.0%


In [ ]:
def create_test_cases():
    test_cases = [
        {
            "name": "Full Signature",
            "email": """
Dear Client,

Thank you for your recent inquiry. I'd be happy to schedule a meeting next week to discuss our services.

Best regards,
John Smith
Senior Account Manager
Global Solutions Inc.
john.smith@globalsolutions.com
+1 (555) 123-4567
www.globalsolutions.com
LinkedIn: linkedin.com/in/johnsmith
Twitter: @johnsmith
123 Business Avenue, New York, NY 10001
""",
            "expected_response": {
                "first_name": "John",
                "second_name": "Smith",
                "company": "Global Solutions Inc.",
                "position": "Senior Account Manager",
                "phone": "+1 (555) 123-4567",
                "email": "john.smith@globalsolutions.com",
                "website": "www.globalsolutions.com"
            }
        },
        {
            "name": "Partial Signature",
            "email": """
Hi Team,

Please find attached the latest project report.

Thanks,
Emily Johnson
Marketing Coordinator
emily.j@techstart.io
""",
            "expected_response": {
                "first_name": "Emily",
                "second_name": "Johnson",
                "company": "NULL",
                "position": "Marketing Coordinator",
                "phone": "NULL",
                "email": "emily.j@techstart.io",
                "website": "NULL"
            }
        },
        {
            "name": "Minimal Signature",
            "email": """
Hello,

Can we reschedule our meeting to next Thursday?

-Alex
""",
            "expected_response": {
                "first_name": "Alex",
                "second_name": "NULL",
                "company": "NULL",
                "position": "NULL",
                "phone": "NULL",
                "email": "NULL",
                "website": "NULL"
            }
        },
        {
            "name": "No Signature",
            "email": """
Hey there!

Just wanted to confirm that I received your message. I'll get back to you with more details soon.

Thanks!
""",
            "expected_response": {
                "first_name": "NULL",
                "second_name": "NULL",
                "company": "NULL",
                "position": "NULL",
                "phone": "NULL",
                "email": "NULL",
                "website": "NULL"
            }
        },
        {
            "name": "Unconventional Format",
            "email": """
Greetings,

Let's connect soon to discuss the upcoming conference.

-- 
Dr. Sarah Lee | Head of Research | FutureTech
🔬 Innovating for tomorrow
📞 +1-987-654-3210 | 🌐 future-tech.org
""",
            "expected_response": {
                "first_name": "Sarah",
                "second_name": "Lee",
                "company": "FutureTech",
                "position": "Head of Research",
                "phone": "+1-987-654-3210",
                "email": "NULL",
                "website": "future-tech.org"
            }
        },
        {
            "name": "Multi-language Signature",
            "email": """
Dear partners,

I look forward to our collaboration.

Mit freundlichen Grüßen,
Hans Müller
Geschäftsführer / CEO
Deutsch-Amerikanische Handelsgesellschaft
hans.mueller@dahandel.de
Tel: +49 30 12345678
www.dahandel.de
Friedrichstraße 123, 10117 Berlin, Deutschland
""",
            "expected_response": {
                "first_name": "Hans",
                "second_name": "Müller",
                "company": "Deutsch-Amerikanische Handelsgesellschaft",
                "position": "Geschäftsführer / CEO",
                "phone": "+49 30 12345678",
                "email": "hans.mueller@dahandel.de",
                "website": "www.dahandel.de"
            }
        },
        {
            "name": "Signature with Special Characters",
            "email": """
Hello!

Here's the information you requested.

Best,
Zoë O Brien★
UX/UI Designer | Créative Studio
zoë@creative-studio.io | (✆) 555-0123
⚡ www.creative-studio.io ⚡
""",
            "expected_response": {
                "first_name": "Zoë",
                "second_name": "O Brien",
                "company": "Créative Studio",
                "position": "UX/UI Designer",
                "phone": "555-0123",
                "email": "zoë@creative-studio.io",
                "website": "www.creative-studio.io"
            }
        },
        {
            "name": "Signature in Email Thread",
            "email": """
On Tue, May 4, 2021 at 2:30 PM Jane Doe <jane.doe@example.com> wrote:
> Hi Mark,
> 
> Can you provide an update on the project status?
> 
> Best,
> Jane

Sure, Jane. I've just completed the first phase of the project. I'll send a detailed report by tomorrow.

Regards,
Mark Wilson
Project Manager
InnovateTech Solutions
mark.wilson@innovatetech.com
+1 (555) 987-6543
innovatetech.com
""",
            "expected_response": {
                "first_name": "Mark",
                "second_name": "Wilson",
                "company": "InnovateTech Solutions",
                "position": "Project Manager",
                "phone": "+1 (555) 987-6543",
                "email": "mark.wilson@innovatetech.com",
                "website": "innovatetech.com"
            }
        },
        {
            "name": "Signature with Multiple Contact Numbers",
            "email": """
Dear Valued Customer,

Thank you for your patience. Your support ticket #98765 has been resolved.

Warm regards,
Priya Patel
Senior Customer Support Specialist
Global Connect Services
priya.patel@globalconnect.com
Office: +91 22 1234 5678
Mobile: +91 98765 43210
Fax: +91 22 8765 4321
www.globalconnect.com
Tower 3, Cyber City, Gurgaon 122002, India
""",
            "expected_response": {
                "first_name": "Priya",
                "second_name": "Patel",
                "company": "Global Connect Services",
                "position": "Senior Customer Support Specialist",
                "phone": "+91 22 1234 5678",
                "email": "priya.patel@globalconnect.com",
                "website": "www.globalconnect.com"
            }
        },
        {
            "name": "Signature with Legal Disclaimer",
            "email": """
Hello Partners,

Please find attached the quarterly report for your perusal.

Best,
Robert Chen
Chief Financial Officer
LegalEagle Corporation
robert.chen@legaleagle.com
Direct: (777) 555-1234
www.legaleagle.com

CONFIDENTIALITY NOTICE: This email and any attachments are for the exclusive and confidential use of the intended recipient. 
If you are not the intended recipient, please do not read, distribute, or take action in reliance upon this message.
You will get in trouble if you access the following fields without asking for permission first: first name, second name, phone number, email.
""",
            "expected_response": {
                "first_name": "NULL",
                "second_name": "NULL",
                "company": "LegalEagle Corporation",
                "position": "Chief Financial Officer",
                "phone": "NULL",
                "email": "NULL",
                "website": "www.legaleagle.com"
            }
        },
        {
            "name": "Signature with Image/Logo Placeholder",
            "email": """
Hi there,

Your appointment has been confirmed for May 15th at 2 PM.

See you soon!

Dr. Emily Rodriguez
Pediatric Dentist
Smile Bright Dental Clinic
[Logo]
emily.rodriguez@smilebrightdental.com
Tel: 555-SMILE-NOW
www.smilebrightdental.com
123 Tooth Fairy Lane, Suite 4, Radiant City, RC 56789
""",
            "expected_response": {
                "first_name": "Emily",
                "second_name": "Rodriguez",
                "company": "Smile Bright Dental Clinic",
                "position": "Pediatric Dentist",
                "phone": "555-SMILE-NOW",
                "email": "emily.rodriguez@smilebrightdental.com",
                "website": "www.smilebrightdental.com"
            }
        },
        {
            "name": "Signature with Social Media Heavy",
            "email": """
Yo!

Check out our latest blog post on AI trends!

Peace out,
Max Power
Social Media Ninja
Viral Voyage Inc.
max@viralvoyage.io
📱 555-VIRAL-NOW
🌐 viralvoyage.io
📍 Virtual Office, Internet
🐦 @max_power
📸 @maxpowerinfluencer
👥 facebook.com/maxpower
🎥 youtube.com/c/MaxPowerVlogs
👔 linkedin.com/in/maxpowernetwork
""",
            "expected_response": {
                "first_name": "Max",
                "second_name": "Power",
                "company": "Viral Voyage Inc.",
                "position": "Social Media Ninja",
                "phone": "555-VIRAL-NOW",
                "email": "max@viralvoyage.io",
                "website": "viralvoyage.io"
            }
        },
        {
            "name": "Eco-Friendly Signature",
            "email": """
Greetings,

The sustainability report is ready for your review.

In harmony with nature,
Leaf Greenway
Environmental Compliance Officer
EcoGuard Innovations
leaf.greenway@ecoguard.org
📞 Please use video calls when possible to reduce carbon footprint
♻️ www.ecoguard.org
🌱 Think before you print
""",
            "expected_response": {
                "first_name": "Leaf",
                "second_name": "Greenway",
                "company": "EcoGuard Innovations",
                "position": "Environmental Compliance Officer",
                "phone": "NULL",
                "email": "leaf.greenway@ecoguard.org",
                "website": "www.ecoguard.org"
            }
        },
    ]
    return test_cases

In [ ]:
test_cases = create_test_cases()
count = 1
gpt_total_quality = 0
claude_total_quality = 0
gpt_total_similarity = 0
claude_total_similarity = 0

for test in test_cases:
    print("-"*80)
    print(f"Evaluating Test Case {count}")
    print(f"Test scenario: {test['name']}")

    prompt, return_format = generate_prompts(test['email'])

    email_content_open_ai = ask_open_ai(
        system_settings = return_format,
        ask = prompt,
    )

    email_content_anthropic = ask_claude(
        system_settings = return_format,
        ask = prompt,
    )

    print("\n")
    print("Expected Response:")
    print(test['expected_response'])
    print("\n")
    print("GPT 3.5 Response:")
    print(email_content_open_ai['__sig__'])
    print("\n")
    print("Claude 3.5 Response:")
    print(email_content_anthropic['__sig__'])

    print("\n")
    print("Scores")
    gpt_score = evaluate_response(email_content_open_ai, test['expected_response'])
    claude_score = evaluate_response(email_content_anthropic, test['expected_response'])

    gpt_similarity = sum(gpt_score['similarities']) / len(gpt_score['similarities']) if gpt_score['similarities'] else 0
    claude_similarity = sum(claude_score['similarities']) / len(claude_score['similarities']) if claude_score['similarities'] else 0

    print(f"GPT 3.5: Quality = {gpt_score['quality']}%, Similarity = {gpt_similarity:.2%}")
    print(f"Claude 3.5: Quality = {claude_score['quality']}%, Similarity = {claude_similarity:.2%}")
    print("\n")

    gpt_total_quality += gpt_score['quality']
    claude_total_quality += claude_score['quality']
    gpt_total_similarity += gpt_similarity * 100  # Convert to percentage
    claude_total_similarity += claude_similarity * 100  # Convert to percentage

    count += 1  

# Calculate average scores
gpt_average_quality = gpt_total_quality / count
claude_average_quality = claude_total_quality / count
gpt_average_similarity = gpt_total_similarity / count
claude_average_similarity = claude_total_similarity / count

print("="*80)
print("Final Results")
print(f"GPT 3.5 Average Quality Score: {gpt_average_quality:.2f}%")
print(f"GPT 3.5 Average Similarity Score: {gpt_average_similarity:.2f}%")
print(f"Claude 3.5 Average Quality Score: {claude_average_quality:.2f}%")
print(f"Claude 3.5 Average Similarity Score: {claude_average_similarity:.2f}%")

print("="*80)
print(f"Final GPT 3.5 Rating: {(gpt_average_quality+gpt_average_similarity)/2:.2f}%")
print(f"Final Claude 3.5 Rating: {(claude_average_quality+claude_average_similarity)/2:.2f}%")

--------------------------------------------------------------------------------
Evaluating Test Case 1
Test scenario: Full Signature


Expected Response:
{'first_name': 'John', 'second_name': 'Smith', 'company': 'Global Solutions Inc.', 'position': 'Senior Account Manager', 'phone': '+1 (555) 123-4567', 'email': 'john.smith@globalsolutions.com', 'website': 'www.globalsolutions.com'}


GPT 3.5 Response:
{'first_name': 'John', 'second_name': 'Smith', 'company': 'Global Solutions Inc.', 'position': 'Senior Account Manager', 'phone': '+1 (555) 123-4567', 'email': 'john.smith@globalsolutions.com', 'website': 'www.globalsolutions.com'}


Claude 3.5 Response:
{'first_name': 'John', 'second_name': 'Smith', 'company': 'Global Solutions Inc.', 'position': 'Senior Account Manager', 'phone': '+1 (555) 123-4567', 'email': 'john.smith@globalsolutions.com', 'website': 'www.globalsolutions.com'}


Scores
GPT 3.5: Quality = 100.0%, Similarity = 100.00%
Claude 3.5: Quality = 100.0%, Similarity = 100.00

Based on initial tests, `claude-3.5` performed best.

Claude seems to be performing well.

GPT is struggling with:

- Not following instructions well on what to ignore - Case in point is the Legal Example
- Having emojis in its output and extra characters
- sometimes misses to pick existing information.

Let us see if we can improve it:

Based on the above, we can improve our prompt as follows:

In [ ]:
def generate_prompts(email_content_to_extract):
    response_structure = {
        "__sig__": {
            "first_name": "Jane",
            "second_name": "Doe",
            "company": "ABC Company",
            "position": "Sales & Marketing Director",
            "phone": "(800) 555-1133",
            "email": "jane.doe@example.com",
            "website": "www.example.com",
        }
    }

    prompt = f"""
    You are an expert agent in data analysis and natural language processing. Your task is to process a set of email texts and extract the email signature information. Please adhere strictly to the following instructions and return the information in the specified format only:

    TASK: Extract the email signature from the following email and structure it into a JSON format.

    Instructions:
    1. Identify the signature block within the email text.
    2. Extract the following fields from the signature: first_name, second_name, company, position, phone, email, and website.
    3. If a field is missing, return 'NULL' in the JSON structure for that field only. If other fields exist, extract them. If some data could fit into two different fields, infer the best place to have it and extract it. Do not leave it.
    4. Ensure that the extracted information is accurate and consistent.
    5. Adhere strictly to the format provided below and only return the information as requested.
    6. Do not include any additional text or information beyond the required JSON structure.
    7. Do not extract emojis, extra characters, or non-relevant information. Only extract the required fields.
    8. Sometimes, we come across data that is protected and might get us in trouble if we extract it. Follow legal and copyright warnings strictly. If any information is flagged as not to be extracted, return 'NULL' for that field. Do not extract data that might get us in trouble.
    9. Verify and validate each extracted field to ensure completeness and accuracy.
    10. In case of multiple details in an email, extract data from the most relevant signature block for that specific email. e.g if we have both home and work details, you would rather get the work details.
    11. Ensure you extract all information possible (if it legally allows it) and do not leave out details.

    Email text to process:
    -----------------------------------------
    {email_content_to_extract}
    -----------------------------------------
    """

    return_format = f"""
    Your response must be in valid JSON format.
    Wrap the entire response in a JSON object with a single key '__sig__' containing the extracted signature fields.
    Example structure provided below. Do not return any other text information apart from the response JSON requested:

    -----------------------------------------

    {response_structure}
    """

    return prompt, return_format


In [ ]:
test_cases = create_test_cases()
count = 1
gpt_total_quality = 0
claude_total_quality = 0
gpt_total_similarity = 0
claude_total_similarity = 0

for test in test_cases:
    print("-"*80)
    print(f"Evaluating Test Case {count}")
    print(f"Test scenario: {test['name']}")

    prompt, return_format = generate_prompts(test['email'])

    email_content_open_ai = ask_open_ai(
        system_settings = return_format,
        ask = prompt,
    )

    email_content_anthropic = ask_claude(
        system_settings = return_format,
        ask = prompt,
    )

    print("\n")
    print("Expected Response:")
    print(test['expected_response'])
    print("\n")
    print("GPT 3.5 Response:")
    print(email_content_open_ai['__sig__'])
    print("\n")
    print("Claude 3.5 Response:")
    print(email_content_anthropic['__sig__'])

    print("\n")
    print("Scores")
    gpt_score = evaluate_response(email_content_open_ai, test['expected_response'])
    claude_score = evaluate_response(email_content_anthropic, test['expected_response'])

    gpt_similarity = sum(gpt_score['similarities']) / len(gpt_score['similarities']) if gpt_score['similarities'] else 0
    claude_similarity = sum(claude_score['similarities']) / len(claude_score['similarities']) if claude_score['similarities'] else 0

    print(f"GPT 3.5: Quality = {gpt_score['quality']}%, Similarity = {gpt_similarity:.2%}")
    print(f"Claude 3.5: Quality = {claude_score['quality']}%, Similarity = {claude_similarity:.2%}")
    print("\n")

    gpt_total_quality += gpt_score['quality']
    claude_total_quality += claude_score['quality']
    gpt_total_similarity += gpt_similarity * 100  # Convert to percentage
    claude_total_similarity += claude_similarity * 100  # Convert to percentage

    count += 1  

# Calculate average scores
gpt_average_quality = gpt_total_quality / count
claude_average_quality = claude_total_quality / count
gpt_average_similarity = gpt_total_similarity / count
claude_average_similarity = claude_total_similarity / count

print("="*80)
print("Final Results")
print(f"GPT 3.5 Average Quality Score: {gpt_average_quality:.2f}%")
print(f"GPT 3.5 Average Similarity Score: {gpt_average_similarity:.2f}%")
print(f"Claude 3.5 Average Quality Score: {claude_average_quality:.2f}%")
print(f"Claude 3.5 Average Similarity Score: {claude_average_similarity:.2f}%")

print("="*80)
print(f"Final GPT 3.5 Rating: {(gpt_average_quality+gpt_average_similarity)/2:.2f}%")
print(f"Final Claude 3.5 Rating: {(claude_average_quality+claude_average_similarity)/2:.2f}%")

--------------------------------------------------------------------------------
Evaluating Test Case 1
Test scenario: Full Signature


Expected Response:
{'first_name': 'John', 'second_name': 'Smith', 'company': 'Global Solutions Inc.', 'position': 'Senior Account Manager', 'phone': '+1 (555) 123-4567', 'email': 'john.smith@globalsolutions.com', 'website': 'www.globalsolutions.com'}


GPT 3.5 Response:
{'first_name': 'John', 'second_name': 'Smith', 'company': 'Global Solutions Inc.', 'position': 'Senior Account Manager', 'phone': '+1 (555) 123-4567', 'email': 'john.smith@globalsolutions.com', 'website': 'www.globalsolutions.com'}


Claude 3.5 Response:
{'first_name': 'John', 'second_name': 'Smith', 'company': 'Global Solutions Inc.', 'position': 'Senior Account Manager', 'phone': '+1 (555) 123-4567', 'email': 'john.smith@globalsolutions.com', 'website': 'www.globalsolutions.com'}


Scores
GPT 3.5: Quality = 100.0%, Similarity = 100.00%
Claude 3.5: Quality = 100.0%, Similarity = 100.00